In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [4]:
train_images=train_images/255.0
test_images=test_images/255.0

In [5]:
train_images[0].shape

(28, 28)

In [6]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [25]:
# def build_model(hp):
#   model=keras.Sequential([
#       keras.layers.Conv2D(
#           filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
#           kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
#           activation='relu',
#           input_shape=(28,28,1)
#       ),
#       keras.layers.Conv2D(
#           filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
#           kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
#           activation='relu'
#       ),
#       keras.layers.Flatten(),
#       keras.layers.Dense(
#           units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
#           activation='relu'
#       ),
#       keras.layers.Dense(10,activation='softmax')
#   ])
#   model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
#                 loss='sparse_categorical_crossentropy',
#                 metrics=['accuracy']
#                 )
#   return model
def build_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
            activation='relu',
            input_shape=(28, 28, 1)
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
            kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
            activation='relu'
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
            activation='relu'
        ),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [11]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.1 MB/s eta 0:00:00


In [26]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters



In [27]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,
                          directory='output',
                          project_name="Mnist Fashion")

Reloading Tuner from output/Mnist Fashion/tuner0.json


In [28]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)


Trial 5 Complete [00h 21m 24s]
val_accuracy: 0.8536666631698608

Best val_accuracy So Far: 0.9081666469573975
Total elapsed time: 00h 57m 00s


In [29]:
model=tuner_search.get_best_models(num_models=1)[0]

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 112)       2912      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        64576     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 80)                2478160   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 2546458 (9.71 MB)
Trainable params: 2546458 (9.71 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 336s 199ms/step - loss: 0.1610 - accuracy: 0.9403 - val_loss: 0.2548 - val_accuracy: 0.9118
Epoch 5/10
1688/1688 [==============================] - 333s 197ms/step - loss: 0.1249 - accuracy: 0.9529 - val_loss: 0.2810 - val_accuracy: 0.9087
Epoch 6/10
1688/1688 [==============================] - 331s 196ms/step - loss: 0.0937 - accuracy: 0.9651 - val_loss: 0.3219 - val_accuracy: 0.9127
Epoch 7/10
1688/1688 [==============================] - 333s 197ms/step - loss: 0.0728 - accuracy: 0.9723 - val_loss: 0.3612 - val_accuracy: 0.9128
Epoch 8/10
1688/1688 [==============================] - 334s 198ms/step - loss: 0.0550 - accuracy: 0.9797 - val_loss: 0.3817 - val_accuracy: 0.9112
Epoch 9/10
1688/1688 [==============================] - 335s 198ms/step - loss: 0.0445 - accuracy: 0.9835 - val_loss: 0.3959 - val_accuracy: 0.9080
Epoch 10/10
1688/1688 [==============================] - 334s 198ms/step - loss: 0.0390 - accuracy: 0.9860 - val